## Driver Safety Prediction


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!kaggle datasets download robinreni/revitsone-5class

Dataset URL: https://www.kaggle.com/datasets/robinreni/revitsone-5class
License(s): DbCL-1.0
 99% 977M/983M [00:11<00:00, 67.8MB/s]
100% 983M/983M [00:11<00:00, 89.0MB/s]


In [ ]:
import zipfile
import os

# Define the path to your downloaded zip file
zip_file_path = '/content/revitsone-5class.zip'  # Change this to the actual path of your zip file

# Define the destination folder for extraction
extraction_folder = '/content/drive/MyDrive/DATA2/'  # Change this if you want to extract to a different folder

# Check if the destination folder exists, if not, create it
if not os.path.exists(extraction_folder):
    os.makedirs(extraction_folder)

# Extract the downloaded zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_folder)

print('Dataset extraction complete.')

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from numpy import asarray
import numpy as np
import pandas as pd
from PIL import Image
import cv2
import glob
import os
import random
import subprocess
import matplotlib.pyplot as plt
from skimage.io import imread
from matplotlib.patches import Rectangle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

In [ ]:
image_list_other = []
image_list_safe = []
image_list_talking = []
image_list_text = []
image_list_turn = []

# Update the base path to where the dataset was extracted
base_path = '/content/drive/MyDrive/DATA2/Revitsone-5classes/'

for other in os.listdir(os.path.join(base_path, "other_activities")):
    if other.endswith(".png") or other.endswith(".jpg"):
        image_list_other.append(os.path.join(base_path, "other_activities", other))
        print(os.path.join(base_path, "other_activities", other))

for safe in os.listdir(os.path.join(base_path, "safe_driving")):
    if safe.endswith(".png") or safe.endswith(".jpg"):
        image_list_safe.append(os.path.join(base_path, "safe_driving", safe))
        print(os.path.join(base_path, "safe_driving", safe))

for talking in os.listdir(os.path.join(base_path, "talking_phone")):
    if talking.endswith(".png") or talking.endswith(".jpg"):
        image_list_talking.append(os.path.join(base_path, "talking_phone", talking))
        print(os.path.join(base_path, "talking_phone", talking))

for text in os.listdir(os.path.join(base_path, "texting_phone")):
    if text.endswith(".png") or text.endswith(".jpg"):
        image_list_text.append(os.path.join(base_path, "texting_phone", text))
        print(os.path.join(base_path, "texting_phone", text))

for turn in os.listdir(os.path.join(base_path, "turning")):
    if turn.endswith(".png") or turn.endswith(".jpg"):
        image_list_turn.append(os.path.join(base_path, "turning", turn))
        print(os.path.join(base_path, "turning", turn))

In [ ]:
# Update the paths to match the actual paths in the lists.
# The following example assumes that the base path is correct
# but the other parts of the path are incorrect

# Corrected paths for image_list_other
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_79.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_4664.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_7973.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_13318.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_13396.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_13541.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_13625.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_20398.jpg')
image_list_other.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/other_activities/img_22266.jpg')

# Corrected paths for image_list_turn
image_list_turn.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/turning/img_8771.jpg')
image_list_turn.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/turning/img_62337.jpg')
image_list_turn.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/turning/img_67523.jpg')
image_list_turn.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/turning/img_70552.jpg')
image_list_turn.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/turning/img_84605.jpg')
image_list_turn.remove('/content/drive/MyDrive/DATA2/Revitsone-5classes/turning/img_101434.jpg')

In [ ]:
font = {'family':'Times New Roman','color':'#1f211f'}
background_color = '#fab72f'

In [ ]:
plt.figure(1, figsize=(15, 9))

# Adjust the position of the title above the subplots
plt.subplots_adjust(top=0.85)  # Adjust the top margin for title spacing
plt.axis('off')

n = 0

# Add the title and position it above the subplots
plt.text(
    0.5, 0.93, "Random images of people who talk with their phone",
    ha='center', va='center', transform=plt.gcf().transFigure,
    fontsize=25,
    bbox=dict(facecolor='lightgray', alpha=0.8)
)

# Loop to add images in the subplots
for i in range(4):
    n += 1
    random_img = random.choice(image_list_talking)
    imgs = imread(random_img)
    plt.subplot(2, 2, n)
    plt.imshow(imgs)
    plt.axis('off')  # Remove axis labels and ticks from subplots

plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()


In [ ]:
from matplotlib.font_manager import FontProperties

plt.figure(1, figsize=(15, 9))
plt.axis('off')
n = 0
for i in range(4):
    n += 1
    random_img = random.choice(image_list_text)
    imgs = imread(random_img)

    # Create a FontProperties object with the desired font
    font_prop = FontProperties(family=font['family'])  # Remove the 'color' argument

    plt.suptitle("Random images of people who text with their phone",
                 fontproperties=font_prop, fontsize=25,
                 backgroundcolor=background_color,
                 color=font['color']) # Add the color argument to suptitle
    plt.subplot(2, 2, n)
    plt.imshow(imgs)

plt.show()

In [ ]:
plt.figure(1, figsize=(15, 9))
plt.axis('off')
n = 0
for i in range(4):

    n += 1
    random_img = random.choice(image_list_turn)
    imgs = imread(random_img)
    # Use font properties directly instead of fontdict
    plt.suptitle("Random images of people who turn around",
                 fontsize=25,
                 fontfamily=font['family'], # Use font['family'] for font family
                 color=font['color'], # Use font['color'] for font color
                 backgroundcolor=background_color)
    plt.subplot(2,2,n)
    plt.imshow(imgs)

plt.show()

In [ ]:
plt.figure(1, figsize=(15, 9))
plt.axis('off')
n = 0
for i in range(4):

    n += 1
    random_img = random.choice(image_list_safe)
    imgs = imread(random_img)
    plt.suptitle("Random images of people who drive safely",
                 fontsize=25,  # Removed 'fontdict' and used individual font properties
                 fontfamily=font['family'],  # Set font family
                 color=font['color'],  # Set font color
                 backgroundcolor=background_color)
    plt.subplot(2,2,n)
    plt.imshow(imgs)

plt.show()

In [ ]:
print("Number of samples in (Class = Other) = " ,len(image_list_other))
print("Number of samples in (Class = Safe Driving) = " ,len(image_list_safe))
print("Number of samples in (Class = Talking Phone) = " ,len(image_list_talking))
print("Number of samples in (Class = Texting Phone) = " ,len(image_list_text))
print("Number of samples in (Class = Turning) = " ,len(image_list_turn))

In [ ]:
print(.75*len(image_list_other) , .2*len(image_list_other) ,.05*len(image_list_other))
print(.75*len(image_list_safe) , .2*len(image_list_safe) ,.05*len(image_list_safe))
print(.75*len(image_list_talking) , .2*len(image_list_talking) ,.05*len(image_list_talking))
print(.75*len(image_list_text) , .2*len(image_list_text) ,.05*len(image_list_text))
print(.75*len(image_list_turn) , .2*len(image_list_turn) ,.05*len(image_list_turn))

In [ ]:
print("Train","Test", "Valid")

train_other = image_list_other[:1589]
test_other = image_list_other[1589:2012]
valid_other = image_list_other[2012:]

print (len(train_other), len(test_other), len(valid_other))

train_safe = image_list_safe[:1652]
test_safe = image_list_safe[1652:2092]
valid_safe = image_list_safe[2092:]

print (len(train_safe), len(test_safe), len(valid_safe))

train_talking = image_list_talking[:1626]
test_talking = image_list_talking[1626:2059]
valid_talking = image_list_talking[2059:]

print (len(train_talking), len(test_talking), len(valid_talking))

train_text = image_list_text[:1652]
test_text = image_list_text[1652:2092]
valid_text = image_list_text[2092:]

print (len(train_text), len(test_text), len(valid_text))

train_turn = image_list_turn[:1547]
test_turn = image_list_turn[1547:1959]
valid_turn = image_list_turn[1959:]

print (len(train_turn), len(test_turn), len(valid_turn))

In [ ]:
train_other_df = pd.DataFrame({'image':train_other, 'label':'Other'})
train_safe_df = pd.DataFrame({'image':train_safe, 'label':'Safe'})
train_talking_df = pd.DataFrame({'image':train_talking, 'label':'Talk'})
train_text_df = pd.DataFrame({'image':train_text, 'label':'Text'})
train_turn_df = pd.DataFrame({'image':train_turn, 'label':'Turn'})
test_other_df = pd.DataFrame({'image':test_other, 'label':'Other'})
test_safe_df = pd.DataFrame({'image':test_safe, 'label':'Safe'})
test_talking_df = pd.DataFrame({'image':test_talking, 'label':'Talk'})
test_text_df = pd.DataFrame({'image':test_text, 'label':'Text'})
test_turn_df = pd.DataFrame({'image':test_turn, 'label':'Turn'})

In [ ]:
valid_other_df = pd.DataFrame({'image':valid_other, 'label':'Other'})
valid_safe_df = pd.DataFrame({'image':valid_safe, 'label':'Safe'})
valid_talking_df = pd.DataFrame({'image':valid_talking, 'label':'Talk'})
valid_text_df = pd.DataFrame({'image':valid_text, 'label':'Text'})
valid_turn_df = pd.DataFrame({'image':valid_turn, 'label':'Turn'})
train_df = pd.concat([train_other_df, train_safe_df, train_talking_df, train_text_df, train_turn_df])
test_df = pd.concat([test_other_df, test_safe_df, test_talking_df, test_text_df, test_turn_df])
val_df = pd.concat([valid_other_df, valid_safe_df, valid_talking_df, valid_text_df, valid_turn_df])

In [ ]:
train_df.head()

In [ ]:
print("Number of rows in train dataframe is: ", len(train_df))
print("Number of rows in test dataframe is: ", len(test_df))
print("Number of rows in val dataframe is: ", len(val_df))

In [ ]:
random_img_height = random.choice(train_other)
image= cv2.imread(random_img_height)

height, width= image.shape[:2]

print("The height is ", height)

print("The width is ", width)

In [ ]:
Batch_size = 64
Img_height = 240
Img_width = 240

In [ ]:
trainGenerator = ImageDataGenerator(rescale=1./255.)
valGenerator = ImageDataGenerator(rescale=1./255.)
testGenerator = ImageDataGenerator(rescale=1./255.)

In [ ]:
trainDataset = trainGenerator.flow_from_dataframe(
  dataframe=train_df,
  class_mode="categorical",
  x_col="image",
  y_col="label",
  batch_size=Batch_size,
  seed=42,
  shuffle=True,
  target_size=(Img_height,Img_width) #set the height and width of the images
)

testDataset = testGenerator.flow_from_dataframe(
  dataframe=test_df,
  class_mode='categorical',
  x_col="image",
  y_col="label",
  batch_size=Batch_size,
  seed=42,
  shuffle=True,
  target_size=(Img_height,Img_width)
)

valDataset = valGenerator.flow_from_dataframe(
  dataframe=val_df,
  class_mode='categorical',
  x_col="image",
  y_col="label",
  batch_size=Batch_size,
  seed=42,
  shuffle=True,
  target_size=(Img_height,Img_width)
)

In [ ]:
def AlexNet():
    inp = layers.Input((240, 240, 3))
    x = layers.Conv2D(96, 11, 4, activation='relu')(inp)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, 2)(x)
    x = layers.Conv2D(256, 5, 1, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(3, 2)(x)
    x = layers.Conv2D(384, 3, 1, activation='relu')(x)
    x = layers.Conv2D(384, 3, 1, activation='relu')(x)
    x = layers.Conv2D(256, 3, 1, activation='relu')(x)
    x = layers.MaxPooling2D(3, 2)(x)
    x = layers.Flatten()(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(4096, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(5, activation='softmax')(x)

    model_Alex = models.Model(inputs=inp, outputs=x)

    return model_Alex

model_Alex = AlexNet()
model_Alex.summary()

In [ ]:
tf.keras.utils.plot_model(
    model_Alex,
    to_file='alex_model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    show_layer_activations=True,
    dpi=100
)

# epochs

In [ ]:
model_Alex.compile(loss=BinaryCrossentropy(),
              optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
Alex_model = model_Alex.fit(trainDataset, epochs=20, validation_data=valDataset)

In [ ]:
training_loss_alex = Alex_model.history['loss']
val_loss_alex = Alex_model.history['val_loss']
training_acc_alex = Alex_model.history['accuracy']
val_acc_alex = Alex_model.history['val_accuracy']
epoch_count = range(1, len(training_loss_alex) + 1)

# Visualize loss history
plt.figure(figsize=(8,3), dpi=200)
plt.plot(epoch_count, training_loss_alex, 'r--', color= 'navy')
plt.plot(epoch_count, val_loss_alex, '--bo',color= 'orangered', linewidth = '2.5', label='line with marker')
plt.legend(['Training Loss', 'Val Loss'])
plt.title('Number of epochs & Loss in ALEXNET')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.xticks(np.arange(1,21,1))
plt.show();

In [ ]:
plt.figure(figsize=(10,5), dpi=200)
plt.plot(epoch_count, training_acc_alex, 'r--', color= 'navy')
plt.plot(epoch_count, val_acc_alex, '--bo',color= 'orangered', linewidth = '2.5', label='line with marker')
plt.legend(['Training Acc', 'Val Acc'])
plt.title('Number of epochs & Accuracy in ALEXNET')
plt.xlabel('Epoch')
plt.ylabel('Acc')
plt.xticks(np.arange(1,21,1))
plt.plot();
plt.show();